In [6]:
import cv2
import numpy as np
import pywt
import numpy as np
from PIL import Image
import pywt
from scipy.linalg import svd, diagsvd, inv
from cryptography.fernet import Fernet
import random

In [68]:
class WatermarkProcessor:
    def __init__(self, arnold_key, fernet_key):
        self.arnold_key = arnold_key
        self.fernet_key = fernet_key
        self.watermark_dimensions = None  # Initialize with None to store watermark dimensions

    def arnold_transform(self, image, iterations=1):
        """Scramble image using Arnold's Cat Map"""
        n_rows, n_cols = image.shape
        transformed_image = np.copy(image)
        for _ in range(iterations):
            new_image = np.zeros_like(transformed_image)
            for x in range(n_rows):
                for y in range(n_cols):
                    new_x = (self.arnold_key[0] * x + self.arnold_key[1] * y) % n_rows
                    new_y = (self.arnold_key[2] * x + self.arnold_key[3] * y) % n_cols
                    new_image[new_x, new_y] = transformed_image[x, y]
            transformed_image = new_image
        return transformed_image

    def arnold_inverse_transform(self, image, iterations=1):
        """Inverse Arnold transform to descramble the image pixels"""
        n_rows, n_cols = image.shape
        transformed_image = np.copy(image)
        for _ in range(iterations):
            new_image = np.zeros_like(transformed_image)
            for x in range(n_rows):
                for y in range(n_cols):
                    new_x = (self.arnold_key[3] * x - self.arnold_key[1] * y) % n_rows
                    new_y = (-self.arnold_key[2] * x + self.arnold_key[0] * y) % n_cols
                    new_image[new_x, new_y] = transformed_image[x, y]
            transformed_image = new_image
        return transformed_image

    def encrypt_coordinates(self, coordinates):
        """Encrypt coordinates using Fernet"""
        fernet = Fernet(self.fernet_key)
        encrypted = fernet.encrypt(str(coordinates).encode())
        return encrypted

    def decrypt_coordinates(self, encrypted_coordinates):
        """Decrypt coordinates using Fernet"""
        fernet = Fernet(self.fernet_key)
        decrypted = fernet.decrypt(encrypted_coordinates).decode()
        return eval(decrypted)

    def apply_dwt_svd(self, block):
        """Apply DWT and then SVD"""
        coeffs = pywt.dwt2(block, 'haar')
        LL, (LH, HL, HH) = coeffs
        U, s, Vh = svd(LL)
        return U, s, Vh, LH, HL, HH

    def reconstruct_image(self, U, s, Vh, LH, HL, HH):
        """Reconstruct image block from SVD and DWT"""
        LL = np.dot(U, np.dot(np.diag(s), Vh))
        coeffs = LL, (LH, HL, HH)
        return pywt.idwt2(coeffs, 'haar')

    def embed_watermark(self, U, V, bit, scaling_factor):
        """Embed a single bit into the SVD matrices U and V"""
        mu = (abs(U[1, 0]) + abs(U[2, 0])) / 2
        if bit == 1:
            U[1, 0], U[2, 0] = mu + scaling_factor / 2, mu - scaling_factor / 2
        else:
            U[1, 0], U[2, 0] = mu - scaling_factor / 2, mu + scaling_factor / 2
        return U, V

    def extract_bit(self, U):
        """Extract a single bit from the SVD U matrix"""
        return 1 if abs(U[1, 0]) > abs(U[2, 0]) else 0

    def pad_image_to_blocksize(self, image, block_size):
        """Pad image to make its dimensions multiples of block size"""
        padded_height = ((image.shape[0] // block_size) + 1) * block_size if image.shape[0] % block_size != 0 else image.shape[0]
        padded_width = ((image.shape[1] // block_size) + 1) * block_size if image.shape[1] % block_size != 0 else image.shape[1]
        padded_image = np.pad(image, ((0, padded_height - image.shape[0]), (0, padded_width - image.shape[1])), mode='constant', constant_values=0)
        return padded_image

    def process_image(self, host_image_path, watermark_image_path, scaling_factor, block_size=8):
        """Process the host image to embed the watermark"""
        host_image = Image.open(host_image_path).convert('L')
        watermark_image = Image.open(watermark_image_path).convert('L')
        watermark = np.array(watermark_image) > 128  # Binarize watermark
        
        self.watermark_dimensions = watermark.shape

        host_image = np.array(host_image)
        host_image = self.pad_image_to_blocksize(host_image, block_size)
        scrambled_watermark = self.arnold_transform(watermark, iterations=5)

        num_blocks = (host_image.shape[0] // block_size) * (host_image.shape[1] // block_size)
        if num_blocks < np.prod(self.watermark_dimensions):
            raise ValueError("Not enough blocks available in the host image to embed the watermark")

        selected_blocks = random.sample(range(num_blocks), np.prod(self.watermark_dimensions))

        encrypted_coords = self.encrypt_coordinates(selected_blocks)
        watermarked_image_blocks = host_image.reshape((host_image.shape[0] // block_size, block_size, -1, block_size)).swapaxes(1, 2).reshape(-1, block_size, block_size)

        watermark_flat = scrambled_watermark.flatten()
        for idx, block in enumerate(watermarked_image_blocks):
            if idx in selected_blocks:
                bit_index = selected_blocks.index(idx)
                bit = watermark_flat[bit_index]
                U, s, Vh, LH, HL, HH = self.apply_dwt_svd(block)
                U, Vh = self.embed_watermark(U, Vh, bit, scaling_factor)
                watermarked_image_blocks[idx] = self.reconstruct_image(U, s, Vh, LH, HL, HH)

        watermarked_image = watermarked_image_blocks.reshape((host_image.shape[0] // block_size, host_image.shape[1] // block_size, block_size, block_size)).swapaxes(1, 2).reshape(host_image.shape)
        watermarked_image = Image.fromarray(watermarked_image.astype(np.uint8))
        watermarked_image.save('watermarked_image.png')
        return encrypted_coords

    def extract_watermark(self, watermarked_image_path, encrypted_coords, block_size=8):
        """Extract the watermark from the watermarked image using stored dimensions"""
        if not self.watermark_dimensions:
            raise ValueError("Watermark dimensions not set. Ensure watermark is processed first.")

        watermarked_image = Image.open(watermarked_image_path).convert('L')
        watermarked_image = np.array(watermarked_image)

        selected_blocks = self.decrypt_coordinates(encrypted_coords)
        watermark_bits = []
        watermarked_image_blocks = watermarked_image.reshape((watermarked_image.shape[0] // block_size, block_size, -1, block_size)).swapaxes(1, 2).reshape(-1, block_size, block_size)

        for idx in selected_blocks:
            block = watermarked_image_blocks[idx]
            U, s, Vh, _, _, _ = self.apply_dwt_svd(block)
            bit = self.extract_bit(U)
            watermark_bits.append(bit)

        if len(watermark_bits) != np.prod(self.watermark_dimensions):
            raise ValueError(f"Extracted bits count {len(watermark_bits)} does not match expected watermark size {self.watermark_dimensions}.")

        watermark_image = np.array(watermark_bits).reshape(self.watermark_dimensions)
        watermark_image = self.arnold_inverse_transform(watermark_image, iterations=5)
        return Image.fromarray(watermark_image.astype(np.uint8) * 255)


In [71]:
# Example usage
arnold_key = [1, 1, 1, 2]  # Arnold transformation parameters
fernet_key = Fernet.generate_key()  # Symmetric encryption key for coordinates
processor = WatermarkProcessor(arnold_key, fernet_key)

encrypted_coords = processor.process_image('/Users/kabir/Downloads/Watermark/images/1003.png', '/Users/kabir/Downloads/Watermark/images/watermark_64x112_image.png',scaling_factor=0.05, block_size=4)


ValueError: Not enough blocks available in the host image to embed the watermark

In [59]:
encrypted_coords

b'gAAAAABmMagEu1uXnHXgQ3ogTGIDsBJPKxa1XphvN4yzV6poKiki6ZdRFF5ssvtfEP1TcRXUVNUOyKcbHQ64P1Ixth2ntOQLCPrZkhhC2URKAe2mGBBekBFjqKGxIulDQV7vA_7qsVmiBIE0XFH3deLp01KdF2YOw6aEpZeA3D20qptgUFfO6t8D20dMzX6oG6nWulJav-1nWeOPb7WOiGiFlksWFMzkP45Aknjvdjnancu5Gow37Y15eMR2shUp3Rbx5ssfSp3ps0PqJSIaqI76lG5ZQzmHGReFnUJR_gf3ZvNMSSSdMgH0SJjnXKHPfWnCzyxYimQR8AVWVLBRwbO0ZBp3TZMbYXJkPNg8v8WgAZd81KpvPd5QlYdwgqbOWEE_fyTg-iALLrmk8AGfIKiswdFRXzLS_OBHDQAWR5lS4kW7UCq7p6i6oxiuXCjNKRnEWiyOp32Fs1ABM1-2AGIOBxE6QIJB41fwfc5lgX61Z0oqZaBGwtisdIfHE282uwBm0MFWz26YVMKWdOgcGkHVBx4abIQJYVSghjuIqcyJJzXpTScAKdXPYhwLyWWbqm3pIVVDH78D1UlK90o02K89qCrKMsazCJcxLCI77H9L5T0kY0uF7mj9H-792QUOZAJEAem3eRV6CBhW2ipHlbD_iXTBAy8Dq10ha8KrEi0Jylc87N2PPD4CGjKP7GzPFL8KIlNiavY4rv9SEbzpUUwJcp3f9FP08Zc4dxtf_3yP2uwdlmNasSa1GBfqxnLwmNlECm6ofDVIkCi64QcC-bzycvZYXy11Sg=='

In [60]:
extracted_watermark = processor.extract_watermark('/Users/kabir/Downloads/Watermark/exps/watermarked_image.png', encrypted_coords,block_size=16)
extracted_watermark.save('/extracted_watermark.png')

ValueError: cannot reshape array of size 130 into shape (112,64)

In [102]:
import numpy as np
import cv2
from cryptography.fernet import Fernet

class SVDWatermarking:
    def __init__(self, host_image_path, watermark_image_path, scaling_factor=1, arnold_key=None, fernet_key=None):
        self.host_image = cv2.imread(host_image_path, cv2.IMREAD_GRAYSCALE)
        self.watermark_image = cv2.imread(watermark_image_path, cv2.IMREAD_GRAYSCALE)
        self.scaling_factor = scaling_factor
        self.arnold_key = arnold_key or 10
        self.fernet_key = fernet_key or Fernet.generate_key()
        self.watermark_dimensions = self.watermark_image.shape
        self.block_size = 8

    def arnold_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (x + y) % w
                    new_y = (x + 2 * y) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        return result

    def arnold_inverse_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (2 * x - y) % w
                    new_y = (x - y) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        return result

    def embed_watermark(self):
        # Preprocess watermark
        watermark = (self.watermark_image > 128).astype(int)
        watermark = self.arnold_transform(watermark, self.arnold_key)

        # Select embedding blocks using HVS
        blocks = [np.hsplit(row, self.host_image.shape[1] // self.block_size) for row in np.vsplit(self.host_image, self.host_image.shape[0] // self.block_size)]
        blocks = [block.astype(np.float32) for row in blocks for block in row]
        dct_blocks = [cv2.dct(block.reshape(-1, 8)).ravel() for block in blocks]
        selected_blocks_indices = np.argsort([np.sum(np.abs(dct_block)) for dct_block in dct_blocks])[:np.prod(self.watermark_dimensions)]
        selected_blocks = [blocks[idx] for idx in selected_blocks_indices]

        watermarked_image = np.copy(self.host_image)
        encrypted_coords = []
        for i, block in enumerate(selected_blocks):
            U, S, V = np.linalg.svd(block)
            watermark_bit = watermark.flat[i]
            if watermark_bit == 1:
                U[1, 0] = np.sign(U[1, 0]) * (np.abs(U[1, 0]) + self.scaling_factor)
                U[2, 0] = np.sign(U[2, 0]) * (np.abs(U[2, 0]) - self.scaling_factor)
            else:
                U[1, 0] = np.sign(U[1, 0]) * (np.abs(U[1, 0]) - self.scaling_factor)
                U[2, 0] = np.sign(U[2, 0]) * (np.abs(U[2, 0]) + self.scaling_factor)
            watermarked_block = np.dot(U, np.dot(np.diag(S), V))
            row, col = np.unravel_index(selected_blocks_indices[i], (self.host_image.shape[0] // self.block_size, self.host_image.shape[1] // self.block_size))
            watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size] = watermarked_block.astype(np.uint8)
            encrypted_coords.append(self.encrypt_coordinates(row, col))

        return watermarked_image, encrypted_coords
    def extract_watermark(self, watermarked_image, encrypted_coords):
        fernet = Fernet(self.fernet_key)
        watermark_bits = []
        for coord in encrypted_coords:
            row, col = self.decrypt_coordinates(coord, fernet)
            block = watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size]
            U, _, _ = np.linalg.svd(block)
            bit = self.extract_bit(U)
            watermark_bits.append(bit)

        watermark_image = np.array(watermark_bits).reshape(self.watermark_dimensions)
        watermark_image = self.arnold_inverse_transform(watermark_image, iterations=self.arnold_key)
        return watermark_image

    def extract_bit(self, U):
        if U[1, 0] > U[2, 0]:
            return 1
        else:
            return 0

    def encrypt_coordinates(self, row, col):
        fernet = Fernet(self.fernet_key)
        coords = f"{row},{col}".encode()
        return fernet.encrypt(coords)

    def decrypt_coordinates(self, encrypted_coords, fernet):
        decoded_coords = fernet.decrypt(encrypted_coords).decode()
        row, col = map(int, decoded_coords.split(","))
        return row, col

In [103]:
arnold_key = 10 # Key for Arnold transform
fernet_key = Fernet.generate_key() # Symmetric encryption key for coordinates

watermarker = SVDWatermarking('/Users/kabir/Downloads/Watermark/images/1002.PNG', '/Users/kabir/Downloads/Watermark/images/watermark_64x112_image.png',arnold_key=arnold_key, fernet_key=fernet_key)
watermarked_image, encrypted_coords = watermarker.embed_watermark()
cv2.imwrite('/watermarked_image_new.png', watermarked_image)

extracted_watermark = watermarker.extract_watermark(watermarked_image,encrypted_coords)
# cv2.imwrite('/extracted_watermark_new.png', extracted_watermark * 255)

In [88]:
watermarked_image

array([[226, 226, 226, ..., 118, 122, 129],
       [226, 226, 226, ..., 143, 138, 148],
       [226, 226, 226, ..., 110, 115, 104],
       ...,
       [210, 210, 210, ..., 184, 182, 180],
       [210, 210, 210, ..., 182, 184, 185],
       [210, 210, 210, ..., 183, 182, 181]], dtype=uint8)

In [104]:
# Ensure that the values are within the 0-255 range for uint8
normalized_watermark = np.clip(extracted_watermark * 255, 0, 255)

# Convert the data type to uint8
watermark_uint8 = normalized_watermark.astype(np.uint8)

# Create the image
image = Image.fromarray(watermark_uint8)
image.show()

In [95]:
extracted_watermark

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [98]:
image.save('extracted_watermark_new.png')